In [ ]:
import os

# os.environ["KERAS_BACKEND"] = 'plaidml.keras.backend'

import keras.backend as K
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D, Dropout, Dense, Input, concatenate, AveragePooling2D, Flatten, Layer

import cv2
import numpy as np
from keras.datasets import cifar10
from keras.utils import np_utils

import math
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
import matplotlib
import matplotlib.pyplot as plt

# Loading dataset and Performing some preprocessing steps.

num_classes = 10


# The CIFAR-10 dataset (Canadian Institute For Advanced Research) is a collection of images that are commonly
# used to train machine learning and computer vision algorithms.
def load_cifar10_data(img_rows, img_cols):

    """
    Cifar Image를 다운로드 받아서
    이미지를 training과 valid로 나누고
    Preprocessing을 해 준다.
    :param img_rows: 리사이징 이미지 크기 (Row)
    :param img_cols: 리사이징 이미지 크기 (Col)
    :return: normalized된 train과 valid 이미지 numpy array
    """

    # x는 feature y는 label
    # load cifar-10 training and validation sets
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
    print(x_train.shape)
    x_train = x_train[0:6000, :, :]
    y_train = y_train[0:6000]
    print(x_train.shape)
    x_test = x_test[6000:8000,  :, :]
    y_test = y_test[6000:8000]

    # resize training images
    x_train = np.array([cv2.resize(img, (img_rows, img_cols)) for img in x_train[:, :, :]])
    x_test = np.array([cv2.resize(img, (img_rows, img_cols)) for img in x_test[:, :, :]])
    print(x_train.shape)
    # transform targets to keras compatible format
    y_train = np_utils.to_categorical(y_train, num_classes)
    y_test = np_utils.to_categorical(y_test, num_classes)
    # One-hot 인코딩 해주는 함수 10진수->2진 binary

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    # convert from integers to floats

    # preprocess data (영상이미지라서 255.0으로 나눠서 normalize한다) normalize to range 0-1
    x_train = x_train / 255.0
    x_test = x_test / 255.0

    return x_train, y_train, x_test, y_test


# define inception v1 architecture
def inception_module(x, filters_1x1, filters_3x3_reduce, filters_3x3, filters_5x5_reduce, filters_5x5,
                     filters_pool_proj, name=None, bias_init='zeros'):
    """
    https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D
    tf.keras.layers.Conv2D(
    filters, kernel_size, strides=(1, 1), padding='valid',
    data_format=None, dilation_rate=(1, 1), groups=1, activation=None,
    use_bias=True, kernel_initializer='glorot_uniform',
    bias_initializer='zeros', kernel_regularizer=None,
    bias_regularizer=None, activity_regularizer=None, kernel_constraint=None,
    bias_constraint=None, **kwargs
    )

    kernel_initializer :
    Initializer for the kernel weights matrix (see keras.initializers). Defaults to 'glorot_uniform'.
    bias_initializer :
    Initializer for the bias vector (see keras.initializers). Defaults to 'zeros'.

    https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D
    tf.keras.layers.MaxPool2D(
    pool_size=(2, 2), strides=None, padding='valid', data_format=None,
    **kwargs

    https://www.tensorflow.org/api_docs/python/tf/keras/layers/concatenate
    tf.keras.layers.concatenate(
    inputs, axis=-1, **kwargs
    axis? https://supermemi.tistory.com/11
    """

    conv_1x1 = Conv2D(filters_1x1, (1, 1), padding='same', activation='relu', bias_initializer=bias_init)(x)

    conv_3x3_reduce = Conv2D(filters_3x3_reduce, (1, 1), padding='same', activation='relu', bias_initializer=bias_init)(x)

    conv_3x3 = Conv2D(filters_3x3, (3, 3), padding='same', activation='relu', bias_initializer=bias_init)(conv_3x3_reduce)

    conv_5x5_reduce = Conv2D(filters_5x5_reduce, (1, 1), padding='same', activation='relu', bias_initializer=bias_init)(x)

    conv_5x5 = Conv2D(filters_5x5, (5, 5), padding='same', activation='relu', bias_initializer=bias_init)(conv_5x5_reduce)

    max_pool = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)

    pool_proj = Conv2D(filters_pool_proj, (1, 1), padding='same', activation='relu', bias_initializer=bias_init)(max_pool)

    output = concatenate([conv_1x1, conv_3x3, conv_5x5, pool_proj], axis=3, name=name)

    return output


def decay(epoch, steps=100):
    initial_lrate = 0.01
    drop = 0.96
    epoch_drop = 8
    lrate = initial_lrate * math.pow(drop, math.floor((1 + epoch) / epoch_drop))
    return lrate
#https://machinelearningmastery.com/using-learning-rate-schedules-deep-learning-models-python-keras/
#더 낮은 loss로 수렴하기 위해서 학습률을 감소시키는 방법을 사용

"""
class LRN2D(Layer):
     This code is adapted from pylearn2.
     License at: https://github.com/lisa-lab/pylearn2/blob/master/LICENSE.txt
    

    def __init__(self, alpha=0.0001,k=1,beta=0.75,n=3, name=None):
        if n % 2 == 0:
            raise NotImplementedError("LRN2D only works with odd n. n provided: " + str(n))
        super(LRN2D, self).__init__()
        self.alpha = alpha
        self.k = k
        self.beta = beta
        self.n = n
        self.name = name

    def get_output(self, train):
        X = self.get_input(train)
        return tf.nn.lrn(X)

    def get_config(self):
        return {"name": self.__class__.__name__,
                "alpha": self.alpha,
                "k": self.k,
                "beta": self.beta,
                "n": self.n}
"""
#LRN 사용예시


x_train, y_train, x_valid, y_valid = load_cifar10_data(224, 224)

"""
https://stackoverflow.com/questions/2480650/what-is-the-role-of-the-bias-in-neural-networks
What is the role of the bias in neural networks?
c
 which may be critical for successful learning.

 The main function of a bias is to provide every node with a trainable constant value (in addition to the normal inputs that the node recieves). 
 You can achieve that with a single bias node with connections to N nodes, or with N bias nodes each with a single connection; the result should be the same.
"""

# 모든값이 특정 상수인 텐서를 생성하는 초기값 설정기
bias_init = keras.initializers.Constant(value=0.2)

input_layer = Input(shape=(224, 224, 1))

# Layer 1
x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu', name='conv_1_7x7/2',
           bias_initializer=bias_init)(input_layer)

x = MaxPool2D((3, 3), strides=(2, 2), name='max_pool_1_3x3/2', padding='same')(x)

# https://taeguu.tistory.com/29
# 실제 모델에서 LRN 사용 한 이유

# Layer 2 
x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu', name='conv2/3x3_reduce',
           bias_initializer=bias_init)(x)

x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu', name='conv_2_3x3/1',
           bias_initializer=bias_init)(x)

x = MaxPool2D((3, 3), strides=(2, 2), name='max_pool_2_3x3/2', padding='same')(x)

# Layer 3
x = inception_module(x, 64, 96, 128, 16, 32, 32, name='inception_3a', bias_init=bias_init)
x = inception_module(x, 128, 128, 192, 32, 96, 64, name='inception_3b', bias_init=bias_init)
x = MaxPool2D((3, 3), strides=(2, 2), name='max_pool_3_3x3/2', padding='same')(x)

# Layer 4
x = inception_module(x, 192, 96, 208, 16, 48, 64, name='inception_4a')

# Layer 4 - Auxiliary Learning 1
x1 = AveragePooling2D((5, 5), strides=3, name='avg_pool_aux_1')(x)
x1 = Conv2D(128, (1, 1), padding='same', activation='relu', name='conv_aux_1')(x1)
x1 = Flatten()(x1)
x1 = Dense(1024, activation='relu', name='dense_aux_1')(x1)
x1 = Dropout(0.7)(x1)
x1 = Dense(10, activation='softmax', name='aux_output_1')(x1)

x = inception_module(x, 160, 112, 224, 24, 64, 64, name='inception_4b', bias_init=bias_init)
x = inception_module(x, 128, 128, 256, 24, 64, 64, name='inception_4c', bias_init=bias_init)
x = inception_module(x, 112, 144, 288, 32, 64, 64, name='inception_4d', bias_init=bias_init)

# Layer 4 - Auxiliary Learning 2
x2 = AveragePooling2D((5, 5), strides=3, name='avg_pool_aux_2')(x)
x2 = Conv2D(128, (1, 1), padding='same', activation='relu', name='conv_aux_2')(x2)
x2 = Flatten()(x2)
x2 = Dense(1024, activation='relu', name='dense_aux_2')(x2)
x2 = Dropout(0.7)(x2)
x2 = Dense(10, activation='softmax', name='aux_output_2')(x2)

x = inception_module(x, 256, 160, 320, 32, 128, 128, name='inception_4e', bias_init=bias_init)
x = MaxPool2D((3, 3), strides=(2, 2), name='max_pool_4_3x3/2', padding='same')(x)

# Layer 5
x = inception_module(x, 256, 160, 320, 32, 128, 128, name='inception_5a', bias_init=bias_init)
x = inception_module(x, 384, 192, 384, 48, 128, 128, name='inception_5b', bias_init=bias_init)
x = AveragePooling2D((7, 7), strides=(1, 1), name='pool5/7x7_s2')(x)
x = Flatten()(x)
x = Dropout(0.4)(x)
# regularization방식 중 하나
x = Dense(10, activation='softmax', name='output')(x)

# input -> output - model
model = Model(input_layer, [x, x1, x2], name='inception_v1')

model.summary()

epoch = 25
initial_lrate = 0.01

# 추출된 데이터에대해 error gradient를 계산, gradient descent 알고리즘을 적용하는 방법
# https://mangkyu.tistory.com/62
sgd = SGD(learning_rate=initial_lrate, momentum=0.9, nesterov=False)
lr_sc = LearningRateScheduler(decay, verbose=1)
# a function that takes an epoch index (integer, indexed from 0) and current learning rate (float) as inputs and returns a new learning rate as output (float).
# 학습률을 조정하기 위한 함수

model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy'],
              loss_weights=[1, 0.3, 0.3], optimizer=sgd, metrics=['accuracy'])
# 분류 할때 label -> 배열 softmax_cross_entropy를 loss로 하고자 할 때
# https://crazyj.tistory.com/153

# 학습시키기
# batch : 몇문항을 풀고 해답을맞추는지
# epoch : 같은문제를 몇번 풀어볼지 (overfitting)
history = model.fit(x_train, [y_train, y_train, y_train], validation_data=(x_valid, [y_valid, y_valid, y_valid]), epochs=epoch, batch_size=20, callbacks=[lr_sc])



In [ ]:
print(history.history.keys())

In [ ]:
# summarize history for accuracy
plt.plot(history.history['output_accuracy'])
plt.plot(history.history['aux_output_1_accuracy'])
plt.plot(history.history['aux_output_2_accuracy'])
plt.plot(history.history['val_output_accuracy'])
plt.plot(history.history['val_aux_output_1_accuracy'])
plt.plot(history.history['val_aux_output_2_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['output_accuracy', 'aux_output_1_accuracy','aux_output_2_accuracy','val_output_accuracy','val_aux_output_1_accuracy','val_aux_output_2_accuracy'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['output_loss'])
plt.plot(history.history['aux_output_1_loss'])
plt.plot(history.history['aux_output_2_loss'])
plt.plot(history.history['val_loss'])
plt.plot(history.history['val_output_loss'])
plt.plot(history.history['val_aux_output_1_loss'])
plt.plot(history.history['val_aux_output_2_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'output_loss','aux_output_1_loss','aux_output_2_loss','val_lossval_loss','val_output_loss','val_aux_output_1_loss','val_aux_output_2_loss'], loc='upper left')
plt.show()